### gensim 라이브러리 모델에서 GridSearchCV 를 이용하는 방법 
- sklean에서 제공하는 GridSearchCV는 모델의 fit(), transform(), predict() 함수를 이용하여 최적의 매개변수의 값들을 찾는 class
- gensim의 모델들은 해당 함수들이 존재하지 않는다. 
- class를 새로 생성하여 fit(), transform(), predict() 함수를 내장한 객체를 생성
- pipeline, kfold, gridsearch를 이용

In [ ]:
# 라이브러리 로드 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import FastText
from konlpy.tag import Komoran
import numpy as np

In [ ]:
# Komoran 형태소 분석기를 이용한 토큰화 함수 정의 
komoran = Komoran()
def tokenize():
    return lambda x : [t for t in komoran.morphs(x)]
tokenizer = tokenize()

In [71]:
# FastText를 fit(), transform()를 내장한 class 정의 
class FastTextVectorizer(BaseEstimator, TransformerMixin):
    # 생성자 함수 -> class가 생성될때 최초로 한번 실행이 되는 함수 
    # class 내부에서 사용할 데이터들을 저장하기 위해 사용
    def __init__(
            self, 
            # FastText에서 사용할 매개변수 
            # 토큰화된 데이터 (2차원 리스트)
            # vector_size : (좌표축의 개수(차원의 수))
            # window : 주변의 체크 할 단어의 개수
            # min_count : 최소 등장 횟수
            # epochs : 반복 학습의 횟수
            # sg : 확률 계산 방식 (주변 단어를 찾을것인가 중심 단어를 찾을것인가)
            # min_n : subword의 최소 개수(글자 수)
            # max_n : subword의 최대 개수(글자 수)
            # worker : 병렬 처리 스레드의 수
            tokenizer = None, 
            vector_size = 100, 
            min_count = 1, 
            window = 3, 
            sg = 1, 
            epochs = 10, 
            min_n = 3, 
            max_n = 6, 
            worker = 1  
    ):
        # class 안에 저장공간을 생성하여 데이터들 저장 
        self.tokenizer = tokenizer
        self.vector_size = vector_size
        self.window = window
        self.sg = sg
        self.epochs = epochs
        self.min_count = min_count
        self.min_n = min_n
        self.max_n = max_n
        self.worker = worker
        # 시드값 고정 
        self.seed = 42

        # 모델이랑 단어 사전이 저장될 빈 공간을 생성 
        self.model = None
        self.voca = None

    # 총 4개의 메서드를 생성 
    # 토큰화 메서드
    def to_token(self, X):
        # 만약에 토큰화 함수가 존재하지 않는다면 공백을 기준으로 문자를 나눠준다. 
        if self.tokenizer is None:
            # s -> 문장 하나
            # t -> 문장 안에서 단어
            return [ [t for t in s.split()] for s in X ]
        return [ self.tokenizer(s) for s in X ]
    
    # 학습 함수 -> fit( X, Y )
    def fit( self, X, Y ):
        # class 내부의 to_token() 함수를 호출
        sentences = self.to_token(X)
        # FastText를 이용하여 sentences들을 학습
        self.model = FastText(
            sentences= sentences, 
            vector_size= self.vector_size, 
            window = self.window, 
            min_count = self.min_count, 
            sg = self.sg, 
            epochs= self.epochs, 
            min_n = self.min_n, 
            max_n = self.max_n, 
            workers= self.worker, 
            seed = self.seed
        )
        # 학습 한 후 학습 데이터에서의 단어 사전 생성 
        self.voca = set(self.model.wv.key_to_index.keys())
        return self
    
    # 문장에서 평균 벡터로 사용할 doc_vec() 함수를 정의 
    def doc_vec(self, tokens):
        # 하나의 문장을 백터화 -> 단위 백터의 평균 
        # self.model -> FastText 모델 
        # wv -> 단어들과 단위 벡터들이 모여있는 dict 형태의 데이터
        vecs = [ self.model.wv[w] for w in tokens if w in self.model.wv ]
        if not vecs:
            # vecs가 존재하지 않은 경우 -> 빈 리스트인 경우
            # 희소행렬을 되돌려준다. 
            v = np.zeros(self.model.vector_size)
        else:
            v = np.mean(vecs, axis=0)
        return v 
    # 변형함수 정의 (tansform 함수)
    def transform(self, X):
        # 토큰화 
        sentences = self.to_token(X)
        # 임베딩
        mat = np.vstack(
            [self.doc_vec(tokens) for tokens in sentences ]
            )
        return mat

In [72]:
# 1차 class 테스트 
X = [
    '상품 품질이 아주 좋다', 
    '배송이 너무 느리다', 
    '포장이 잘되어 있고 만족합니다',
    '환불이 오래 걸려서 불안하다',
    '가격 대비 만족스럽다', 
    '연락이 잘 안 되고 불친절하다', 
    '디자인이 이쁘고 마음에 든다', 
    '설명과 달라서 실망이다', 
    '배송이 빠르고 기사분이 친절했습니다', 
    '색상이 사진과 달라서 만족하지 못했습니다'
]
Y = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

In [73]:
vec_data = FastTextVectorizer(tokenizer= tokenizer)

In [74]:
vec_data.fit(X, Y)

,tokenizer,<function tok...001FCEEA3F9C0>
,vector_size,100
,min_count,1
,window,3
,sg,1
,epochs,10
,min_n,3
,max_n,6
,worker,1


In [ ]:
# tranform 함수 테스트 
vec_data.transform(X)

In [76]:
# 파이프라인 생성 -> FastTextVectorizer객체와 머신러닝 모델 합친다. 

pipe = Pipeline(
    [
        ('ft', FastTextVectorizer(tokenizer=tokenizer)), 
        ('clf', LogisticRegression(max_iter=2000))
    ]
)

In [ ]:
pipe.fit(X, Y)

In [78]:
# 최적의 파라미터를 찾기 위해 GridSearch와 Fold화 
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [79]:
# 매개변수의 경우들을 dict 데이터로 생성 
grid_params = {
    'ft__vector_size' : [50, 100], 
    'ft__sg' : [0, 1], 
    'clf__C' : [1.0, 2.0]
}

In [80]:
# pipe와 cv, grid_params들을 이용하여 GridSearchCV를 사용
grid = GridSearchCV(
    estimator= pipe, 
    param_grid= grid_params, 
    cv = cv, 
    scoring= 'f1_macro',
    verbose=1
)

In [81]:
# gridsearch 실행 
grid.fit(X, Y)
# 최적의 파라미터를 출력
print("GridSearch 기준 최적의 파라미터 : ", grid.best_params_)
print("GridSearch 의 최적의 F1 : ", grid.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
GridSearch 기준 최적의 파라미터 :  {'clf__C': 1.0, 'ft__sg': 0, 'ft__vector_size': 50}
GridSearch 의 최적의 F1 :  0.39999999999999997


### 연습 
- tokenizer, FastTextVectorizer, StratifiedKfold, Pipeline, GridSearchCV을 이용하여 data 폴더 안에 ratings_train.txt 파일의 데이터를 학습하여 최적의 파라미터를 구하라
    1. train 데이터를 ratings_train.txt에서 상위 500개의 데이터를 이용
    2. test 데이터는 하위 100개의 데이터 
    3. 파라미터 경우의 수
        - FastText
            - window -> 3, 5
            - sg -> 0, 1
            - epochs -> 10, 50
        - Logistic
            - C -> 1.0, 2.0
    4. Fold는 3개를 생성
    5. 최적의 파라미터를 생성
    6. 최적의 모델을 이용하여 test 데이터를 예측한 뒤  class_report 생성

In [114]:
import pandas as pd

In [116]:
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [117]:
# 사용할 컬럼을 제외하고 제거 -> id 컬럼
df.drop('id', axis=1, inplace=True)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  149995 non-null  object
 1   label     150000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [123]:
# document 컬럼에서 문자열 좌우의 공백을 제거 
df['document'] = df['document'].astype(str)

In [ ]:
# map() 함수를 이용해서 원소 각각을 대입하여 변환
df['document'].map(
    lambda x : x.strip()
)

In [128]:
# Series에서 문자열 Method을 사용할수 있는 형태로 변환하고 문자열 함수 사용
df['document'] = df['document'].str.strip()

In [130]:
# document에서 중복된 데이터를 제거 
df.drop_duplicates('document', inplace=True)

In [151]:
# train, test 데이터를 나눠준다. 
X_train = df['document'].head(1000).values
Y_train = df['label'].head(1000).values
X_test = df['document'].tail(100).values
Y_test = df['label'].tail(100).values

In [152]:
# 파이프라인 생성 
pipe2 = Pipeline(
    [
        ('ft', FastTextVectorizer(tokenizer=tokenizer)), 
        ('clf', LogisticRegression(max_iter=2000))
    ]
)

In [153]:
cv2 = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [154]:
grid_params2 = {
    'ft__window' : [3, 5], 
    'ft__sg' : [0, 1], 
    'ft__epochs' : [10, 50], 
    'clf__C' : [1.0, 2.0]
}

In [155]:
grid2 = GridSearchCV(
    estimator=pipe2, 
    cv = cv2, 
    param_grid=grid_params2, 
    scoring= 'f1_macro', 
    verbose=1
)

In [156]:
grid2.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


,estimator,Pipeline(step..._iter=2000))])
,param_grid,"{'clf__C': [1.0, 2.0], 'ft__epochs': [10, 50], 'ft__sg': [0, 1], 'ft__window': [3, 5]}"
,scoring,'f1_macro'
,n_jobs,None
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,tokenizer,<function tok...001FCEEA3F9C0>


In [157]:
# 최적의 스코어 
print(grid2.best_score_)
# 최적의 파라미터
print(grid2.best_params_)

0.7015991575414494
{'clf__C': 2.0, 'ft__epochs': 50, 'ft__sg': 1, 'ft__window': 3}


In [158]:
best_model = grid2.best_estimator_

In [159]:
best_pred = best_model.predict(X_test)

In [160]:
print(classification_report(best_pred, Y_test))

              precision    recall  f1-score   support

           0       0.70      0.72      0.71        46
           1       0.75      0.74      0.75        54

    accuracy                           0.73       100
   macro avg       0.73      0.73      0.73       100
weighted avg       0.73      0.73      0.73       100



In [161]:
# 최적의 파라미터를 이용하여 재학습 -> 예측 검증 
params = grid2.best_params_

In [162]:
pipe3 = Pipeline(
    [
        ('ft', FastTextVectorizer(
            tokenizer= tokenizer, 
            window = params['ft__window'], 
            sg = params['ft__sg'], 
            epochs= params['ft__epochs']
        )), 
        ('clf', LogisticRegression(max_iter=2000,
                                   C = params['clf__C']))
    ]
)

In [163]:
pipe3.fit(X_train, Y_train)

,steps,"[('ft', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,tokenizer,<function tok...001FCEEA3F9C0>
,vector_size,100
,min_count,1
,window,3
,sg,1
,epochs,50
,min_n,3


In [164]:
pred3 = pipe3.predict(X_test)

In [165]:
print(classification_report(pred3, Y_test))

              precision    recall  f1-score   support

           0       0.70      0.72      0.71        46
           1       0.75      0.74      0.75        54

    accuracy                           0.73       100
   macro avg       0.73      0.73      0.73       100
weighted avg       0.73      0.73      0.73       100

